<a href="https://colab.research.google.com/github/Vopiyo/First-Repository/blob/main/LBW_CDSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Configure the page
st.set_page_config(
    page_title="LBW Risk Prediction CDSS",
    page_icon="👶",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for beautiful styling
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
        font-weight: bold;
    }
    .risk-high {
        background-color: #ff6b6b;
        color: white;
        padding: 15px;
        border-radius: 10px;
        font-size: 1.5rem;
        text-align: center;
        font-weight: bold;
        margin: 10px 0;
    }
    .risk-medium {
        background-color: #ffd93d;
        color: black;
        padding: 15px;
        border-radius: 10px;
        font-size: 1.5rem;
        text-align: center;
        font-weight: bold;
        margin: 10px 0;
    }
    .risk-low {
        background-color: #6bcf7f;
        color: white;
        padding: 15px;
        border-radius: 10px;
        font-size: 1.5rem;
        text-align: center;
        font-weight: bold;
        margin: 10px 0;
    }
    .recommendation-box {
        background-color: #f0f2f6;
        padding: 20px;
        border-radius: 10px;
        border-left: 5px solid #1f77b4;
        margin: 10px 0;
    }
    .critical-recommendation {
        background-color: #ffe6e6;
        padding: 20px;
        border-radius: 10px;
        border-left: 5px solid #ff6b6b;
        margin: 10px 0;
    }
    .feature-importance {
        background-color: white;
        padding: 15px;
        border-radius: 10px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
    .sidebar .sidebar-content {
        background-color: #f8f9fa;
    }
</style>
""", unsafe_allow_html=True)

class LBWPredictor:
    def __init__(self):
        self.model = None
        self.preprocessor = None
        self.feature_names = [
            'Pregnancy_Months', 'Mother_height', 'Mother_weight',
            'Preceding_birth_interval(months)', 'MalFansidar_Frequecy',
            'Mother_Age', 'Educational_attainment_No_Eductaion',
            'Residence_Urban', 'Days_Tablets_taken', 'Partner_Age',
            'Maternal_Dewormer_No', 'Wealth_index_Poorer', 'ANC_Visits',
            'ANC1_Timing', 'Worked_Last_12months_Yes', 'Birth_order_number',
            'Worked_Last_12months_No', 'Occupation_Unskilled_Manual',
            'Partner_Education.1', 'Sex_child', 'Iron_Supplements'
        ]

    def initialize_model(self):
        """Initialize a demo Random Forest model - Replace with your trained model in production"""
        # Define feature types based on your original analysis
        numeric_features = [
            'Pregnancy_Months', 'Mother_height', 'Mother_weight',
            'Preceding_birth_interval(months)', 'MalFansidar_Frequecy',
            'Mother_Age', 'Days_Tablets_taken', 'Partner_Age', 'ANC_Visits',
            'ANC1_Timing', 'Birth_order_number', 'Partner_Education.1',
            'Sex_child', 'Iron_Supplements'
        ]

        categorical_features = [
            'Educational_attainment_No_Eductaion', 'Residence_Urban',
            'Maternal_Dewormer_No', 'Wealth_index_Poorer',
            'Worked_Last_12months_Yes', 'Worked_Last_12months_No',
            'Occupation_Unskilled_Manual'
        ]

        # Create preprocessing pipelines
        numeric_transformer = Pipeline(steps=[
            ('scaler', StandardScaler())
        ])

        categorical_transformer = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])

        self.preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)
            ])

        # Create a demo Random Forest model (replace with your trained model)
        self.model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            class_weight='balanced'
        )

        # For demo purposes, we'll create synthetic training data
        # In production, you would load your actual trained model
        np.random.seed(42)
        n_samples = 1000
        demo_data = {}

        for feature in self.feature_names:
            if feature in numeric_features:
                if 'Age' in feature:
                    demo_data[feature] = np.random.randint(18, 45, n_samples)
                elif 'height' in feature:
                    demo_data[feature] = np.random.randint(150, 180, n_samples)
                elif 'weight' in feature:
                    demo_data[feature] = np.random.randint(45, 90, n_samples)
                elif 'ANC_Visits' in feature:
                    demo_data[feature] = np.random.randint(0, 10, n_samples)
                elif 'Iron_Supplements' in feature:
                    demo_data[feature] = np.random.randint(0, 120, n_samples)
                else:
                    demo_data[feature] = np.random.normal(0, 1, n_samples)
            else:
                demo_data[feature] = np.random.randint(0, 2, n_samples)

        X_demo = pd.DataFrame(demo_data)
        y_demo = np.random.randint(0, 2, n_samples)

        # Fit the preprocessor and model
        X_processed = self.preprocessor.fit_transform(X_demo)
        self.model.fit(X_processed, y_demo)

    def predict_risk(self, input_data):
        """Predict LBW risk and return probability and risk level"""
        if self.model is None:
            self.initialize_model()

        try:
            # Create DataFrame with correct feature names
            input_df = pd.DataFrame([input_data])

            # Ensure all features are present
            for feature in self.feature_names:
                if feature not in input_df.columns:
                    input_df[feature] = 0

            # Reorder columns to match training
            input_df = input_df[self.feature_names]

            # Preprocess and predict
            X_processed = self.preprocessor.transform(input_df)
            probability = self.model.predict_proba(X_processed)[0][1]

            # Determine risk level based on thresholds
            if probability >= 0.7:
                risk_level = "High"
            elif probability >= 0.4:
                risk_level = "Medium"
            else:
                risk_level = "Low"

            return probability, risk_level

        except Exception as e:
            st.error(f"Prediction error: {e}")
            return 0.5, "Medium"

    def get_recommendations(self, risk_level, input_data):
        """Generate case-based intervention recommendations"""
        recommendations = []

        # ANC Visit based recommendations
        anc_visits = input_data.get('ANC_Visits', 0)
        if anc_visits < 4:
            recommendations.append({
                'type': 'Critical',
                'title': 'Increase ANC Visits',
                'description': f'Current ANC visits ({anc_visits}) are below recommended 4+ visits. Schedule additional antenatal care appointments.',
                'action': 'Refer to ANC clinic for immediate scheduling of additional visits',
                'timeline': 'Immediate'
            })

        # Iron supplements based recommendations
        iron_supplements = input_data.get('Iron_Supplements', 0)
        if iron_supplements < 60:
            recommendations.append({
                'type': 'Critical',
                'title': 'Iron Supplementation',
                'description': f'Only {iron_supplements} days of iron supplements taken. Aim for 90+ days during pregnancy.',
                'action': 'Prescribe iron supplements (60mg elemental iron + 400μg folic acid daily)',
                'timeline': 'Immediate'
            })

        # Risk-level specific recommendations
        if risk_level == "High":
            recommendations.extend([
                {
                    'type': 'Critical',
                    'title': 'High-Risk Pregnancy Management',
                    'description': 'Patient identified as high risk for LBW. Requires intensive monitoring and specialist care.',
                    'action': 'Refer to specialist obstetric care and schedule bi-weekly monitoring',
                    'timeline': 'Within 1 week'
                },
                {
                    'type': 'Critical',
                    'title': 'Comprehensive Nutritional Support',
                    'description': 'Implement comprehensive nutritional supplementation program with close monitoring.',
                    'action': 'Provide balanced diet plan, multiple micronutrient supplements, and nutritional counseling',
                    'timeline': 'Immediate'
                },
                {
                    'type': 'Important',
                    'title': 'Fetal Growth Monitoring',
                    'description': 'Enhanced fetal surveillance needed for high-risk pregnancy.',
                    'action': 'Schedule ultrasound for fetal growth assessment and Doppler studies',
                    'timeline': 'Within 2 weeks'
                }
            ])

        elif risk_level == "Medium":
            recommendations.extend([
                {
                    'type': 'Important',
                    'title': 'Enhanced Monitoring',
                    'description': 'Moderate risk detected. Requires closer follow-up and monitoring.',
                    'action': 'Schedule additional ANC visits every 2 weeks and fetal growth monitoring',
                    'timeline': 'Within 2 weeks'
                },
                {
                    'type': 'Important',
                    'title': 'Lifestyle Counseling',
                    'description': 'Provide comprehensive health education on pregnancy care.',
                    'action': 'Counsel on nutrition, rest, danger signs in pregnancy, and birth preparedness',
                    'timeline': 'Next visit'
                }
            ])

        else:  # Low risk
            recommendations.append({
                'type': 'Routine',
                'title': 'Standard ANC Care',
                'description': 'Continue with routine antenatal care and monitoring.',
                'action': 'Maintain scheduled ANC visits and routine supplementation',
                'timeline': 'As scheduled'
            })

        # Additional condition-based recommendations
        mother_weight = input_data.get('Mother_weight', 50)
        if mother_weight < 50:
            recommendations.append({
                'type': 'Important',
                'title': 'Nutritional Rehabilitation',
                'description': f'Low maternal weight ({mother_weight}kg) detected. Nutritional support required.',
                'action': 'Implement high-calorie, high-protein diet with nutritional supplements. Target weight gain: 12.5-18kg',
                'timeline': 'Immediate'
            })

        pregnancy_months = input_data.get('Pregnancy_Months', 6)
        if pregnancy_months < 3:
            recommendations.append({
                'type': 'Important',
                'title': 'Early Pregnancy Care Optimization',
                'description': 'Early pregnancy stage - crucial period for interventions.',
                'action': 'Initiate early ANC registration, baseline investigations, and early supplementation',
                'timeline': 'Immediate'
            })

        # Malaria prevention
        mal_fansidar = input_data.get('MalFansidar_Frequecy', 0)
        if mal_fansidar < 2 and pregnancy_months > 3:
            recommendations.append({
                'type': 'Important',
                'title': 'Malaria Prevention',
                'description': 'Insufficient malaria prophylaxis in endemic area.',
                'action': 'Provide IPTp with SP as per national guidelines',
                'timeline': 'Next visit'
            })

        return recommendations

def create_input_form():
    """Create the user input form in sidebar"""
    st.sidebar.header("👶 Patient Information")

    with st.sidebar.form("patient_form"):
        # Demographics Section
        st.subheader("Demographics")
        mother_age = st.number_input("Mother's Age (years)", min_value=15, max_value=50, value=25, help="Age of the pregnant woman")
        mother_height = st.number_input("Mother's Height (cm)", min_value=140, max_value=200, value=160, help="Height in centimeters")
        mother_weight = st.number_input("Mother's Weight (kg)", min_value=40, max_value=120, value=60, help="Current weight in kilograms")

        # Pregnancy Details Section
        st.subheader("Pregnancy Details")
        pregnancy_months = st.slider("Pregnancy Months", min_value=1, max_value=9, value=6, help="Current gestation in months")
        anc_visits = st.number_input("ANC Visits Completed", min_value=0, max_value=20, value=4, help="Number of antenatal care visits attended")
        anc1_timing = st.number_input("First ANC Timing (months)", min_value=1, max_value=9, value=3, help="Gestational age at first ANC visit")
        birth_order = st.number_input("Birth Order Number", min_value=1, max_value=10, value=1, help="Parity - which pregnancy is this?")

        # Key Interventions Section
        st.subheader("Key Interventions")
        iron_supplements = st.number_input("Iron Supplement Days", min_value=0, max_value=180, value=30,
                                         help="Number of days iron supplements were taken during pregnancy")
        days_tablets_taken = st.number_input("Other Medication Days", min_value=0, max_value=180, value=30, help="Days of other medications taken")
        mal_fansidar_freq = st.number_input("Malaria Prevention Doses", min_value=0, max_value=10, value=2, help="Number of malaria prophylaxis doses received")

        # Additional Medical History
        st.subheader("Medical History")
        preceding_interval = st.number_input("Preceding Birth Interval (months)", min_value=0, max_value=120, value=24, help="Interval since last birth")
        partner_age = st.number_input("Partner's Age (years)", min_value=15, max_value=70, value=30, help="Age of the partner")

        # Socioeconomic Factors
        st.subheader("Socioeconomic Factors")
        education = st.selectbox("Educational Attainment",
                               ["No Education", "Primary", "Secondary", "Higher"],
                               help="Highest education level completed")
        residence = st.selectbox("Residence", ["Urban", "Rural"], help="Place of residence")
        wealth_index = st.selectbox("Wealth Index",
                                  ["Poorest", "Poorer", "Middle", "Richer", "Richest"],
                                  help="Household wealth quintile")
        worked_last_year = st.selectbox("Worked Last 12 Months", ["Yes", "No"], help="Employment status")
        occupation = st.selectbox("Occupation",
                                ["Professional", "Skilled Manual", "Unskilled Manual", "Not Working"],
                                help="Current occupation")
        maternal_dewormer = st.selectbox("Received Dewormer", ["Yes", "No"], help="Deworming treatment received")

        # Child Information
        sex_child = st.selectbox("Child's Sex", ["Male", "Female"], help="Biological sex of the baby")
        partner_education = st.number_input("Partner's Education Years", min_value=0, max_value=20, value=8, help="Partner's years of education")

        submitted = st.form_submit_button("🚀 Predict LBW Risk")

    if submitted:
        # Prepare input data for model
        input_data = {
            'Pregnancy_Months': pregnancy_months,
            'Mother_height': mother_height,
            'Mother_weight': mother_weight,
            'Preceding_birth_interval(months)': preceding_interval,
            'MalFansidar_Frequecy': mal_fansidar_freq,
            'Mother_Age': mother_age,
            'Educational_attainment_No_Eductaion': 1 if education == "No Education" else 0,
            'Residence_Urban': 1 if residence == "Urban" else 0,
            'Days_Tablets_taken': days_tablets_taken,
            'Partner_Age': partner_age,
            'Maternal_Dewormer_No': 1 if maternal_dewormer == "No" else 0,
            'Wealth_index_Poorer': 1 if wealth_index in ["Poorest", "Poorer"] else 0,
            'ANC_Visits': anc_visits,
            'ANC1_Timing': anc1_timing,
            'Worked_Last_12months_Yes': 1 if worked_last_year == "Yes" else 0,
            'Birth_order_number': birth_order,
            'Worked_Last_12months_No': 1 if worked_last_year == "No" else 0,
            'Occupation_Unskilled_Manual': 1 if occupation == "Unskilled Manual" else 0,
            'Partner_Education.1': partner_education,
            'Sex_child': 1 if sex_child == "Male" else 0,
            'Iron_Supplements': iron_supplements
        }

        return input_data

    return None

def display_risk_assessment(probability, risk_level):
    """Display the risk assessment results"""
    st.markdown('<div class="main-header">LBW Risk Assessment</div>', unsafe_allow_html=True)

    # Create columns for layout
    col1, col2, col3 = st.columns([1, 2, 1])

    with col2:
        # Risk level display
        risk_probability = probability * 100

        if risk_level == "High":
            st.markdown(f'<div class="risk-high">🚨 HIGH RISK: {risk_probability:.1f}% Probability</div>', unsafe_allow_html=True)
            st.warning("Immediate interventions required. Refer to specialist care.")
        elif risk_level == "Medium":
            st.markdown(f'<div class="risk-medium">⚠️ MEDIUM RISK: {risk_probability:.1f}% Probability</div>', unsafe_allow_html=True)
            st.info("Enhanced monitoring and interventions recommended.")
        else:
            st.markdown(f'<div class="risk-low">✅ LOW RISK: {risk_probability:.1f}% Probability</div>', unsafe_allow_html=True)
            st.success("Continue routine antenatal care.")

        # Probability gauge chart
        fig = go.Figure(go.Indicator(
            mode = "gauge+number+delta",
            value = risk_probability,
            domain = {'x': [0, 1], 'y': [0, 1]},
            title = {'text': "LBW Probability %", 'font': {'size': 24}},
            delta = {'reference': 50, 'increasing': {'color': "red"}},
            gauge = {
                'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': "darkblue"},
                'bar': {'color': "darkblue"},
                'bgcolor': "white",
                'borderwidth': 2,
                'bordercolor': "gray",
                'steps': [
                    {'range': [0, 30], 'color': "lightgreen"},
                    {'range': [30, 70], 'color': "yellow"},
                    {'range': [70, 100], 'color': "red"}
                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 90
                }
            }
        ))

        fig.update_layout(
            height=300,
            margin=dict(l=50, r=50, t=100, b=50)
        )
        st.plotly_chart(fig, use_container_width=True)

def display_recommendations(recommendations):
    """Display intervention recommendations"""
    st.header("🎯 Clinical Recommendations & Interventions")

    # Group recommendations by type
    critical_recs = [r for r in recommendations if r['type'] == 'Critical']
    important_recs = [r for r in recommendations if r['type'] == 'Important']
    routine_recs = [r for r in recommendations if r['type'] == 'Routine']

    if critical_recs:
        st.subheader("🚨 Critical Interventions (Immediate Action Required)")
        for i, rec in enumerate(critical_recs, 1):
            with st.container():
                st.markdown(f"""
                <div class="critical-recommendation">
                    <h4>#{i}: {rec['title']}</h4>
                    <p><strong>Rationale:</strong> {rec['description']}</p>
                    <p><strong>Action Required:</strong> {rec['action']}</p>
                    <p><strong>Timeline:</strong> {rec['timeline']}</p>
                </div>
                """, unsafe_allow_html=True)

    if important_recs:
        st.subheader("📋 Important Interventions")
        for i, rec in enumerate(important_recs, 1):
            with st.container():
                st.markdown(f"""
                <div class="recommendation-box">
                    <h4>#{i}: {rec['title']}</h4>
                    <p><strong>Rationale:</strong> {rec['description']}</p>
                    <p><strong>Action Required:</strong> {rec['action']}</p>
                    <p><strong>Timeline:</strong> {rec['timeline']}</p>
                </div>
                """, unsafe_allow_html=True)

    if routine_recs:
        st.subheader("💚 Routine Care")
        for rec in routine_recs:
            with st.container():
                st.markdown(f"""
                <div class="recommendation-box">
                    <h4>{rec['title']}</h4>
                    <p><strong>Description:</strong> {rec['description']}</p>
                    <p><strong>Action:</strong> {rec['action']}</p>
                </div>
                """, unsafe_allow_html=True)

def display_feature_importance():
    """Display feature importance visualization"""
    st.header("📊 Key Risk Factors Analysis")

    # Feature importance data based on your original analysis
    feature_importance_data = {
        'Feature': [
            'Pregnancy Duration', 'Maternal Height', 'Maternal Weight',
            'Birth Interval', 'Malaria Prevention', 'Maternal Age',
            'No Education', 'Urban Residence', 'Medication Days',
            'Partner Age', 'No Deworming', 'Poor Wealth Status',
            'ANC Visits', 'First ANC Timing', 'Employment Status',
            'Birth Order', 'Iron Supplements', 'Child Sex',
            'Partner Education', 'Occupation Type'
        ],
        'Importance': [
            0.92, 0.83, 0.72, 0.56, 0.50, 0.45,
            0.44, 0.40, 0.38, 0.38, 0.36, 0.35,
            0.33, 0.33, 0.30, 0.28, 0.25, 0.24,
            0.26, 0.28
        ],
        'Category': [
            'Pregnancy', 'Maternal', 'Maternal', 'Obstetric', 'Medical',
            'Maternal', 'Socioeconomic', 'Socioeconomic', 'Medical',
            'Maternal', 'Medical', 'Socioeconomic', 'Intervention',
            'Intervention', 'Socioeconomic', 'Obstetric', 'Intervention',
            'Neonatal', 'Socioeconomic', 'Socioeconomic'
        ]
    }

    df_importance = pd.DataFrame(feature_importance_data)
    df_importance = df_importance.sort_values('Importance', ascending=True)

    # Create colored bars by category
    fig = px.bar(df_importance,
                 x='Importance',
                 y='Feature',
                 orientation='h',
                 title='Feature Importance in LBW Risk Prediction',
                 color='Category',
                 color_discrete_map={
                     'Pregnancy': '#1f77b4',
                     'Maternal': '#ff7f0e',
                     'Intervention': '#2ca02c',
                     'Medical': '#d62728',
                     'Socioeconomic': '#9467bd',
                     'Obstetric': '#8c564b',
                     'Neonatal': '#e377c2'
                 })

    fig.update_layout(
        height=600,
        showlegend=True,
        xaxis_title="Normalized Importance Score",
        yaxis_title="Features",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    st.plotly_chart(fig, use_container_width=True)

def display_anc_iron_analysis(input_data):
    """Display specific analysis for ANC and Iron supplements"""
    if input_data:
        st.header("📈 ANC & Iron Supplement Analysis")

        col1, col2 = st.columns(2)

        with col1:
            anc_visits = input_data.get('ANC_Visits', 0)
            st.metric("ANC Visits", anc_visits,
                     delta="Optimal" if anc_visits >= 4 else "Suboptimal",
                     delta_color="normal" if anc_visits >= 4 else "off")

            # ANC progress bar
            st.write("ANC Visit Progress (Target: 4+ visits)")
            anc_progress = min(anc_visits / 4, 1.0)
            st.progress(anc_progress)

        with col2:
            iron_days = input_data.get('Iron_Supplements', 0)
            st.metric("Iron Supplement Days", iron_days,
                     delta="Adequate" if iron_days >= 90 else "Inadequate",
                     delta_color="normal" if iron_days >= 90 else "off")

            # Iron progress bar
            st.write("Iron Supplement Progress (Target: 90+ days)")
            iron_progress = min(iron_days / 90, 1.0)
            st.progress(iron_progress)

def main():
    """Main application function"""
    # Initialize predictor
    predictor = LBWPredictor()

    # Header
    st.markdown('<div class="main-header">Low Birth Weight Risk Prediction CDSS</div>', unsafe_allow_html=True)
    st.markdown("### Clinical Decision Support System for Healthcare Workers in Kenya")

    # Get user input
    input_data = create_input_form()

    if input_data:
        # Make prediction
        with st.spinner('🔍 Analyzing patient data and predicting LBW risk...'):
            probability, risk_level = predictor.predict_risk(input_data)
            recommendations = predictor.get_recommendations(risk_level, input_data)

        # Display results in tabs
        tab1, tab2, tab3, tab4 = st.tabs(["Risk Assessment", "Interventions", "Risk Factors", "Patient Summary"])

        with tab1:
            display_risk_assessment(probability, risk_level)
            display_anc_iron_analysis(input_data)

        with tab2:
            display_recommendations(recommendations)

        with tab3:
            display_feature_importance()

        with tab4:
            st.header("📋 Patient Summary")

            # Create summary cards
            col1, col2, col3 = st.columns(3)

            with col1:
                st.subheader("Demographics")
                st.write(f"**Age:** {input_data.get('Mother_Age', 'N/A')} years")
                st.write(f"**Height:** {input_data.get('Mother_height', 'N/A')} cm")
                st.write(f"**Weight:** {input_data.get('Mother_weight', 'N/A')} kg")
                st.write(f"**Education:** {'No Education' if input_data.get('Educational_attainment_No_Eductaion', 0) == 1 else 'Educated'}")

            with col2:
                st.subheader("Pregnancy Details")
                st.write(f"**Gestation:** {input_data.get('Pregnancy_Months', 'N/A')} months")
                st.write(f"**ANC Visits:** {input_data.get('ANC_Visits', 'N/A')}")
                st.write(f"**First ANC:** {input_data.get('ANC1_Timing', 'N/A')} months")
                st.write(f"**Birth Order:** {input_data.get('Birth_order_number', 'N/A')}")

            with col3:
                st.subheader("Interventions")
                st.write(f"**Iron Days:** {input_data.get('Iron_Supplements', 'N/A')}")
                st.write(f"**Malaria Doses:** {input_data.get('MalFansidar_Frequecy', 'N/A')}")
                st.write(f"**Medication Days:** {input_data.get('Days_Tablets_taken', 'N/A')}")
                st.write(f"**Deworming:** {'No' if input_data.get('Maternal_Dewormer_No', 0) == 1 else 'Yes'}")

            # Show raw data in expander
            with st.expander("View Complete Patient Data"):
                st.json(input_data)

    else:
        # Welcome message when no data submitted
        st.info("👈 **Please fill out the patient information form in the sidebar to get started.**")

        # Display overview and instructions
        col1, col2 = st.columns([2, 1])

        with col1:
            st.header("Welcome to LBW Risk Prediction CDSS")
            st.markdown("""
            This Clinical Decision Support System helps healthcare workers in Kenya:

            ### 🎯 Key Features:
            - **Risk Prediction**: Machine learning-based LBW risk assessment
            - **Case-Based Recommendations**: Personalized intervention strategies
            - **ANC Optimization**: Focus on antenatal care quality and timing
            - **Iron Supplementation**: Monitoring and recommendations for anemia prevention
            - **Evidence-Based**: Built on validated predictive models

            ### 📊 Model Performance:
            - **Accuracy**: 88.1%
            - **Sensitivity**: 91.2%
            - **Specificity**: 84.5%
            - **ROC AUC**: 0.957

            ### 🚀 How to Use:
            1. Fill patient information in the sidebar
            2. Click 'Predict LBW Risk'
            3. Review risk assessment and recommendations
            4. Implement suggested interventions
            """)

        with col2:
            st.header("Quick Stats")
            st.metric("Model Accuracy", "88.1%")
            st.metric("Sensitivity", "91.2%")
            st.metric("Specificity", "84.5%")
            st.metric("ROC AUC Score", "0.957")

        # Display feature importance on welcome screen
        st.header("Key Predictive Factors")
        display_feature_importance()

if __name__ == "__main__":
    main()

2025-10-05 14:56:53.158 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 14:56:53.161 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 14:56:53.637 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-05 14:56:53.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 14:56:53.641 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 14:56:53.656 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 14:56:53.660 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn